# Simulation of User Preferences

In [1]:
import os
import pandas as pd
from vigor import generate_graphs, label_graphs, predicates, learn_predicates, compute_metrics

## Generate graphs

1. Generate graphs using the fast_gnp_random_graph function from networkx
2. Calculate statistics for each graph

In [2]:
file_path = '../data/generated_graphs_example'

if not os.path.exists(f'{file_path}.csv'):
    print('Generating graph data')
    graphs = generate_graphs(100, 2, 200)
    graphs.to_csv(f'{file_path}.csv', index=False)
else:
    print('Loading graph data')
    graphs = pd.read_csv(f'{file_path}.csv')

Loading graph data


In [3]:
graphs.head()

,graph_type,is_directed_int,has_spatial_attributes,has_temporal_attributes,is_bipartite,n_components,avg_betweenness_centrality,avg_closeness_centrality,avg_eigenvector_centrality,avg_degree,...,eccentricity_avg,n_nodes,node_types,node_attributes,number_of_isolates,density,edge_types,edge_attributes,n_parallel_edges,n_self_loops
0,4,1,0,1,0,2,0.006843,0.504436,0.080592,17.356164,...,3.000000,146,3,6,0,0.119698,5,2,2,5
1,4,0,0,0,0,7,0.011514,0.554609,0.114805,15.666667,...,2.736111,72,1,2,0,0.220657,4,12,0,4
2,4,1,0,1,0,9,0.034914,0.568343,0.198937,6.916667,...,2.666667,24,4,7,0,0.300725,5,15,3,1
3,4,0,0,0,1,1,0.007891,0.561783,0.097868,22.376238,...,2.356436,101,1,4,0,0.223762,4,1,0,0
4,4,0,1,1,1,5,0.004791,0.659691,0.094859,52.600000,...,2.000000,110,1,0,0,0.482569,3,0,0,4


## Sample designers

We evaluate the ability of VIGOR to recover the rules that were used to represent simulated users. We design 3 versions of this simulated user: bob_informed, who follows the rules 100\% of the time; bob_semi_informed, who follows the rules 75\% of the time and chooses other visualizations randomly the other 25\% of the time; and bob_uninformed who follows the rules 50\% of the time and chooses other visualizations randomly the other 50\% of the time.

In [4]:
informed = label_graphs(graphs, predicates, conformance=1)
semi_informed = label_graphs(graphs, predicates, conformance=0.75)
uninformed = label_graphs(graphs, predicates, conformance=0.5)

In [5]:
informed

0       MATRIX
1      PAOHVIS
2     NODETRIX
3       MATRIX
4      PAOHVIS
        ...   
94      MATRIX
95      MATRIX
96      MATRIX
97     PAOHVIS
98      MATRIX
Length: 99, dtype: object

## Learning predicates from labeled data

In [6]:
learned_predicates_informed = learn_predicates(graphs, informed, 1000)
learned_predicates_semi_informed = learn_predicates(graphs, semi_informed, 1000)
learned_predicates_uninformed = learn_predicates(graphs, uninformed, 1000)

Learning predicates for MATRIX
OPTIMIZING [[1 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 1 0 0 0 1 0 1 0 0
  0 1 1 1 1 0 0 1 0 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1
  0 1 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 1 1 1 0 1]] tensor([[ 0.4091,  0.9854, -1.1843,  ..., -0.2397,  1.4005, -0.0255],
        [ 0.4091, -0.9657, -1.1843,  ..., -1.3433,  0.8105,  0.7791],
        [ 0.4091,  0.9854, -1.1843,  ...,  0.3121, -0.9595, -1.6355],
        ...,
        [ 0.4091, -0.9657,  0.8029,  ...,  0.3121, -0.3695,  0.3360],
        [ 0.4091, -0.9657,  0.8029,  ...,  0.3121, -1.5495,  1.3198],
        [ 0.4091,  0.9854, -1.1843,  ...,  1.4158,  0.2205,  0.4699]]) tensor([[       inf, 1.0600e+00,        inf, 1.0596e+00,        inf, 1.3689e+00,
         1.2881e+01, 5.4176e+00, 3.2139e+00, 2.5505e+01, 1.1044e+01, 2.8744e+00,
         2.8950e+00, 1.5461e+03,        inf, 1.1609e+01, 2.4514e+00,        inf,
         3.8501e+01,        inf, 1.0911e+01, 1.4907e+00, 1.5310e+00, 2.2921

RuntimeError: all elements of input should be between 0 and 1

## Comparing learned predicates to initial predicates

### Informed User

In [ ]:
metrics, mean_iou, mean_deviation, inclusion_ratio = compute_metrics(predicates, learned_predicates_informed)

print(f"Mean IoU: {mean_iou:.2f}")
print(f"Mean Deviation: {mean_deviation:.2f}")
print(f"Inclusion Ratio: {inclusion_ratio:.2f}")

### Semi-Informed User

In [ ]:
metrics, mean_iou, mean_deviation, inclusion_ratio = compute_metrics(predicates, learned_predicates_semi_informed)

print(f"Mean IoU: {mean_iou:.2f}")
print(f"Mean Deviation: {mean_deviation:.2f}")
print(f"Inclusion Ratio: {inclusion_ratio:.2f}")

### Unformed User

In [ ]:
metrics, mean_iou, mean_deviation, inclusion_ratio = compute_metrics(predicates, learned_predicates_uninformed)

print(f"Mean IoU: {mean_iou:.2f}")
print(f"Mean Deviation: {mean_deviation:.2f}")
print(f"Inclusion Ratio: {inclusion_ratio:.2f}")